In [1]:
# Installs
#!conda install -c conda-forge geopy --yes #Geolocation library
#!conda install -c conda-forge geocoder --yes #Geolocation library
#!conda install -c conda-forge folium=0.5.0 --yes #Mapping library

In [2]:
# Imports
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import json # library to handle JSON files

from zipfile import ZipFile # library for handling ZIP files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from requests import Request

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
from folium import plugins

print('Libraries imported.')

Libraries imported.


### Download Dutch demographic data based on postal code

Dutch postal codes have 6 positions: four digits followed by two letters. The four digits are the best Dutch proxy of neighbourhoods.

The Dutch Central Bureau of Statistics has publised an Excel dataset of demographics per four digit postal code.

See https://www.cbs.nl/nl-nl/maatwerk/2016/51/bevolking-per-viercijferige-postcode-op-1-januari-2016

In [3]:
#Table of Dutch inhabitants per four digit postal code. Inhabitants split per age and sex, migration background and household composition.
#Excel has multi row header, so header is set to none. Columns BU and BV not imported as we do not need it
demo = pd.read_excel('https://www.cbs.nl/-/media/_excel/2016/51/bevolking-per-postcode-1-januari-2016.xls',
                     sheet_name = 'Bevolking per postcode', header = None, skiprows = 5, nrows = 4037, dtype = float, usecols = range(0, 72))
demo.columns = ['postcode', 'tot_inwoners',
                'tot_0-5', 'tot_5-10', 'tot_10-15', 'tot_15-20', 'tot_20-25', 'tot_25-30', 'tot_30-35', 'tot_35-40', 'tot_40-45', 'tot_45-50',
                'tot_50-55', 'tot_55-60', 'tot_60-65', 'tot_65-70', 'tot_70-75', 'tot_75-80', 'tot_80-85', 'tot_85-90', 'tot_90-95', 'tot_95-',
                'tot_man',
                'man_0-5', 'man_5-10', 'man_10-15', 'man_15-20', 'man_20-25', 'man_25-30', 'man_30-35', 'man_35-40', 'man_40-45', 'man_45-50',
                'man_50-55', 'man_55-60', 'man_60-65', 'man_65-70', 'man_70-75', 'man_75-80', 'man_80-85', 'man_85-90', 'man_90-95', 'man_95-',
                'tot_vrouw',
                'vrouw_0-5', 'vrouw_5-10', 'vrouw_10-15', 'vrouw_15-20', 'vrouw_20-25', 'vrouw_25-30', 'vrouw_30-35', 'vrouw_35-40', 'vrouw_40-45', 'vrouw_45-50',
                'vrouw_50-55', 'vrouw_55-60', 'vrouw_60-65', 'vrouw_65-70', 'vrouw_70-75', 'vrouw_75-80', 'vrouw_80-85', 'vrouw_85-90', 'vrouw_90-95', 'vrouw_95-',
                'tot_allochtoon', 'allochtoon_westers', 'allochtoon_nietwesters',
                'tot_huishoudens', 'eenspersoons', 'meer_zonder_kinderen', 'meer_met_kinderen',
                'gem_huish_grootte'               ]
demo['postcode'] = demo['postcode'].astype(int)
demo.head()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,9605.0,295.0,290.0,220.0,315.0,885.0,1100.0,835.0,650.0,...,25.0,5.0,3770.0,2490.0,1280.0,6325.0,4105.0,1360.0,860.0,1.51
1,1012,8245.0,220.0,135.0,115.0,160.0,1100.0,1705.0,1165.0,700.0,...,5.0,0.0,3795.0,2530.0,1270.0,5975.0,4265.0,1275.0,440.0,1.38
2,1013,21185.0,1110.0,855.0,675.0,690.0,1290.0,2075.0,2145.0,1910.0,...,40.0,10.0,9285.0,4300.0,4985.0,12640.0,7485.0,2630.0,2520.0,1.66
3,1014,580.0,30.0,15.0,5.0,15.0,25.0,135.0,125.0,60.0,...,0.0,0.0,225.0,135.0,85.0,295.0,135.0,115.0,45.0,1.74
4,1015,14860.0,575.0,465.0,430.0,445.0,1085.0,1605.0,1445.0,1105.0,...,20.0,5.0,5395.0,3695.0,1700.0,9850.0,6500.0,1955.0,1390.0,1.51


In [4]:
demo.shape

(4037, 72)

In [5]:
demo.dtypes

postcode                    int64
tot_inwoners              float64
tot_0-5                   float64
tot_5-10                  float64
tot_10-15                 float64
tot_15-20                 float64
tot_20-25                 float64
tot_25-30                 float64
tot_30-35                 float64
tot_35-40                 float64
tot_40-45                 float64
tot_45-50                 float64
tot_50-55                 float64
tot_55-60                 float64
tot_60-65                 float64
tot_65-70                 float64
tot_70-75                 float64
tot_75-80                 float64
tot_80-85                 float64
tot_85-90                 float64
tot_90-95                 float64
tot_95-                   float64
tot_man                   float64
man_0-5                   float64
man_5-10                  float64
man_10-15                 float64
man_15-20                 float64
man_20-25                 float64
man_25-30                 float64
man_30-35     

In [6]:
demo.describe()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
count,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,...,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000,4037.000000
mean,5675.431756,4205.483032,217.184791,230.355462,245.973495,252.276443,264.036413,265.252663,252.012633,250.016101,...,17.548922,4.478573,929.333664,410.021055,519.244488,1912.280159,719.728759,553.768888,638.679713,2.312747
std,2671.752148,4198.150829,243.444008,252.998869,259.059261,257.089459,359.385678,347.647781,307.972293,283.131502,...,24.795854,7.524776,1583.838690,543.553199,1166.762322,2012.592483,982.593115,532.601287,660.946912,0.326196
min,1011.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3332.000000,670.000000,30.000000,35.000000,40.000000,45.000000,35.000000,30.000000,30.000000,30.000000,...,0.000000,0.000000,45.000000,30.000000,10.000000,275.000000,70.000000,95.000000,105.000000,2.150000
50%,5711.000000,2665.000000,125.000000,135.000000,155.000000,165.000000,130.000000,125.000000,125.000000,140.000000,...,5.000000,0.000000,260.000000,175.000000,75.000000,1120.000000,315.000000,370.000000,395.000000,2.330000
75%,8051.000000,6885.000000,345.000000,365.000000,380.000000,395.000000,370.000000,390.000000,395.000000,400.000000,...,25.000000,5.000000,1185.000000,625.000000,510.000000,3125.000000,1050.000000,915.000000,1025.000000,2.500000
max,9999.000000,28315.000000,2125.000000,2320.000000,2395.000000,1855.000000,3665.000000,3130.000000,2720.000000,2155.000000,...,215.000000,70.000000,19785.000000,4380.000000,17385.000000,13200.000000,7995.000000,2635.000000,5595.000000,4.170000


In [7]:
#Convert all demographic data on number of inhabitants to fraction of total inhabitants in postal code
demo.iloc[:,2:67] = demo.iloc[:,2:67].div(demo.tot_inwoners, axis=0)
demo.head()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,9605.0,0.030713,0.030193,0.022905,0.032795,0.092140,0.114524,0.086934,0.067673,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,4105.0,1360.0,860.0,1.51
1,1012,8245.0,0.026683,0.016374,0.013948,0.019406,0.133414,0.206792,0.141298,0.084900,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,4265.0,1275.0,440.0,1.38
2,1013,21185.0,0.052396,0.040359,0.031862,0.032570,0.060892,0.097947,0.101251,0.090158,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,7485.0,2630.0,2520.0,1.66
3,1014,580.0,0.051724,0.025862,0.008621,0.025862,0.043103,0.232759,0.215517,0.103448,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,135.0,115.0,45.0,1.74
4,1015,14860.0,0.038694,0.031292,0.028937,0.029946,0.073015,0.108008,0.097241,0.074361,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,6500.0,1955.0,1390.0,1.51


In [8]:
#Convert all demographic data on number of households to fraction of total households in postal code
demo.iloc[:,68:71] = demo.iloc[:,68:71].div(demo.tot_huishoudens, axis=0)
demo.head()

,postcode,tot_inwoners,tot_0-5,tot_5-10,tot_10-15,tot_15-20,tot_20-25,tot_25-30,tot_30-35,tot_35-40,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,9605.0,0.030713,0.030193,0.022905,0.032795,0.092140,0.114524,0.086934,0.067673,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51
1,1012,8245.0,0.026683,0.016374,0.013948,0.019406,0.133414,0.206792,0.141298,0.084900,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38
2,1013,21185.0,0.052396,0.040359,0.031862,0.032570,0.060892,0.097947,0.101251,0.090158,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66
3,1014,580.0,0.051724,0.025862,0.008621,0.025862,0.043103,0.232759,0.215517,0.103448,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74
4,1015,14860.0,0.038694,0.031292,0.028937,0.029946,0.073015,0.108008,0.097241,0.074361,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51


### Download geolocation data

This website offers datasets for geolocation per Dutch four digit postal codes.
https://git.tuxm.nl/tuxmachine/postcodes/src/4329c858db24b79523fd3fbbaf2df138ccaf16cd

Credit: https://git.tuxm.nl/tuxmachine/postcodes/src/master/README.md

License: https://git.tuxm.nl/tuxmachine/postcodes/src/master/LICENSE

In [9]:
#Four digit postal code LatLong in CSV dataset for Netherlands areas in LatLong geolocation
neighlatlong = pd.read_csv('https://git.tuxm.nl/tuxmachine/postcodes/raw/master/4pp.csv')
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1,1000,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.336243,4.869444,Postbus
1,2,1001,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus
2,3,1002,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
3,4,1003,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
4,5,1005,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus


In [10]:
neighlatlong.shape

(4699, 10)

Drop all areas with category ('Soort') Postbus (=P.O. Box) or Onbekend (=Unkown)

In [11]:
#Drop all rows with Soort is Onbekend
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Onbekend'].index, inplace=True)
#Drop all rows with Soort is Postbus
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Postbus'].index, inplace=True)
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


Drop all cities not being the cities we want to research: Amsterdam, Rotterdam, The Hague and Utrecht

In [12]:
# List of cities to select
cityselect = ['amsterdam','rotterdam','den haag','utrecht']
neighlatlong = neighlatlong[neighlatlong.woonplaats.str.contains('|'.join(cityselect), case=False)]
neighlatlong

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres
14,15,1016,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres
15,16,1017,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres
16,17,1018,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.368387,4.918091,Adres
17,18,1019,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372858,4.938828,Adres
19,20,1021,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.384278,4.920920,Adres


In [13]:
#Oops, we have a city called Nieuw-Amsterdam completely somewhere else in the Netherlands. Drop that.
neighlatlong.drop(neighlatlong[neighlatlong.woonplaats == 'Nieuw-Amsterdam'].index, inplace=True)
neighlatlong

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres
14,15,1016,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.371147,4.883705,Adres
15,16,1017,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364185,4.890775,Adres
16,17,1018,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.368387,4.918091,Adres
17,18,1019,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372858,4.938828,Adres
19,20,1021,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.384278,4.920920,Adres


In [14]:
neighlatlong.shape

(252, 10)

In [15]:
neighlatlong.dtypes

id                              int64
postcode                        int64
woonplaats                     object
alternatieve_schrijfwijzen     object
gemeente                       object
provincie                      object
netnummer                       int64
latitude                      float64
longitude                     float64
soort                          object
dtype: object

In [16]:
neighlatlong.describe()

,id,postcode,netnummer,latitude,longitude
count,252.000000,252.000000,252.000000,252.000000,252.000000
mean,715.353175,2367.206349,31.269841,52.131220,4.681036
std,491.045853,964.835282,22.879700,0.176728,0.306865
min,10.000000,1011.000000,10.000000,51.871742,4.218229
25%,77.750000,1086.750000,10.000000,51.965608,4.421880
50%,817.500000,2571.500000,20.000000,52.085449,4.665768
75%,1062.250000,3068.250000,30.000000,52.344597,4.937426
max,1356.000000,3585.000000,70.000000,52.423834,5.164979


### CBS income data per postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on standardised income based on postal code.
See https://www.cbs.nl/nl-nl/maatwerk/2017/15/besteedbaar-inkomen-per-postcodegebied-2004-2014

In [17]:
#Excel has multi row header, so header is set to none. Only columns for latest year (2014) are imported.
#Columns are: postcode for first four digits postal code, gem_inkomen is average income, std_inkomen is standardised income corrected for householdsize
inkomen = pd.read_excel('https://www.cbs.nl/-/media/_excel/2017/15/besteedbaar-inkomen-postcode-2004-2014.xlsx',
                        sheet_name = 'Tabel1', header = None, usecols = [0, 23, 35], skiprows = 10, skipfooter = 5)
inkomen.columns = ['postcode', 'gem_inkomen', 'std_inkomen' ]
inkomen.head()

,postcode,gem_inkomen,std_inkomen
0,1011,32,26.3
1,1012,26.2,22.4
2,1013,30.3,24.1
3,1014,35,28.5
4,1015,33.3,27.2


In [18]:
#Check data types
inkomen.dtypes

postcode        int64
gem_inkomen    object
std_inkomen    object
dtype: object

In [19]:
#Two income columns have converted to Object instead of Float due to fields containing - instead of zero. Correct this
inkomen['gem_inkomen'] = pd.to_numeric(inkomen['gem_inkomen'], errors='coerce')
inkomen['gem_inkomen'].fillna(0, inplace=True)
inkomen['std_inkomen'] = pd.to_numeric(inkomen['std_inkomen'], errors='coerce')
inkomen['std_inkomen'].fillna(0, inplace=True)
inkomen.head()

,postcode,gem_inkomen,std_inkomen
0,1011,32.0,26.3
1,1012,26.2,22.4
2,1013,30.3,24.1
3,1014,35.0,28.5
4,1015,33.3,27.2


In [20]:
inkomen.shape

(4053, 3)

In [21]:
inkomen.dtypes

postcode         int64
gem_inkomen    float64
std_inkomen    float64
dtype: object

In [22]:
inkomen.describe()

,postcode,gem_inkomen,std_inkomen
count,4053.000000,4053.000000,4053.000000
mean,5673.362941,32.684160,22.438959
std,2671.635711,14.333122,9.528407
min,1011.000000,0.000000,0.000000
25%,3331.000000,30.000000,21.800000
50%,5709.000000,36.400000,25.000000
75%,8045.000000,40.700000,27.200000
max,9999.000000,108.800000,76.800000


### CBS names of areas and neighbourhoods based on postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on areas and neighbourhoods based on postal code.

See: https://www.cbs.nl/nl-nl/maatwerk/2018/36/buurt-wijk-en-gemeente-2018-voor-postcode-huisnummer

In [23]:
#Download and extract ZIP file of CBS area and neighbourhood data
!wget -q -O '2018-cbs-pc6huisnr20180801_buurt-vs2.zip' https://www.cbs.nl/-/media/_excel/2018/36/2018-cbs-pc6huisnr20180801_buurt%20-vs2.zip
with ZipFile('2018-cbs-pc6huisnr20180801_buurt-vs2.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
print('Data downloaded!')

Data downloaded!


In [24]:
#Create dataframe on lowlevel postal code data
pc = pd.read_csv('pc6hnr20180801_gwb-vs2.csv', sep=';', encoding='latin_1')
pc.rename(columns = {'Buurt2018': 'buurtcode', 'Wijk2018': 'wijkcode', 'Gemeente2018': 'gemeentecode'}, inplace = True)
pc.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode
0,1011AB,105,3630400,36304,363
1,1011AB,106,3630400,36304,363
2,1011AB,107,3630400,36304,363
3,1011AB,110,3630400,36304,363
4,1011AB,112,3630400,36304,363


In [25]:
#Create dataframe with neighbourhood codes and names
wijk = pd.read_csv('wijknaam2018.csv', sep=';', encoding='latin_1')
wijk.rename(columns = {'GWBcode8': 'wijkcode', 'GWBlabel': 'wijk'}, inplace = True)
wijk.head()

,wijkcode,wijk
0,300,Wijk 00
1,500,Wijk 00
2,900,Wijk 00 West
3,901,Wijk 01 Oost
4,1000,Wijk 00 Stad


In [26]:
#Create dataframe with area codes and names
buurt = pd.read_csv('buurtnaam2018.csv', sep=';', encoding='latin_1')
buurt.rename(columns = {'GWBcode8': 'buurtcode', 'GWBlabel': 'buurt'}, inplace = True)
buurt.head()

,buurtcode,buurt
0,30000,Appingedam-Centrum
1,30001,Appingedam-West
2,30002,Appingedam-Oost
3,30007,Verspreide huizen Damsterdiep en Eemskanaal
4,30008,Verspreide huizen ten zuiden van Eemskanaal


In [27]:
#Merge postal code dataframe with neighbourhood and area names
pcmerge = pd.merge(pc, wijk, on='wijkcode')
pcmerge = pd.merge(pcmerge, buurt, on='buurtcode')
pcmerge.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode,wijk,buurt
0,1011AB,105,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,106,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,107,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,110,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,112,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland


In [28]:
#Drop columns we will not need: housenumber (too low level), Municipality (will be added by geocode data by name instead of code) and the codes
pcmerge.drop(['Huisnummer', 'buurtcode', 'wijkcode', 'gemeentecode'], axis=1, inplace = True)
pcmerge.head()

,PC6,wijk,buurt
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland


In [29]:
#Create column with first four digits of postal code as all six is too low level
pcmerge['postcode'] = pcmerge['PC6'].str[:4]
pcmerge.head()

,PC6,wijk,buurt,postcode
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011


In [30]:
#Drop PC6 and group on postcode
postcode = pcmerge.groupby(['postcode'], as_index=False).first()
postcode.drop(['PC6'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.
2,1013,Haarlemmerbuurt,Westerdokseiland
3,1014,Westelijk Havengebied,Alfa-driehoek
4,1015,Grachtengordel-West,Langestraat e.o.


In [31]:
postcode.dtypes

postcode    object
wijk        object
buurt       object
dtype: object

In [32]:
#Cast column postcode to INT instead of OBJECT
postcode.postcode = postcode.postcode.astype('int64')
postcode.dtypes

postcode     int64
wijk        object
buurt       object
dtype: object

In [33]:
#Now merge with long/lat
postcode = pd.merge(postcode, neighlatlong, on='postcode')
postcode.head()

,postcode,wijk,buurt,id,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,10,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,11,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
2,1013,Haarlemmerbuurt,Westerdokseiland,12,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
3,1014,Westelijk Havengebied,Alfa-driehoek,13,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
4,1015,Grachtengordel-West,Langestraat e.o.,14,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


In [34]:
#Now merge with demographic data
postcode = pd.merge(postcode, demo, on='postcode')
postcode.head()

,postcode,wijk,buurt,id,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,10,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,11,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38
2,1013,Haarlemmerbuurt,Westerdokseiland,12,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66
3,1014,Westelijk Havengebied,Alfa-driehoek,13,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74
4,1015,Grachtengordel-West,Langestraat e.o.,14,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51


In [35]:
#Drop unneeded columns
postcode.drop(['id', 'alternatieve_schrijfwijzen', 'netnummer', 'soort'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,...,vrouw_90-95,vrouw_95-,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9605.0,0.030713,...,0.002603,0.000521,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8245.0,0.026683,...,0.000606,0.000000,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21185.0,0.052396,...,0.001888,0.000472,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,580.0,0.051724,...,0.000000,0.000000,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14860.0,0.038694,...,0.001346,0.000336,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51


In [36]:
#Now merge with income data
postcode = pd.merge(postcode, inkomen, on='postcode')
postcode.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,latitude,longitude,tot_inwoners,tot_0-5,...,tot_allochtoon,allochtoon_westers,allochtoon_nietwesters,tot_huishoudens,eenspersoons,meer_zonder_kinderen,meer_met_kinderen,gem_huish_grootte,gem_inkomen,std_inkomen
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.372976,4.903957,9605.0,0.030713,...,0.392504,0.259240,0.133264,6325.0,0.649012,0.215020,0.135968,1.51,32.0,26.3
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,52.373386,4.894064,8245.0,0.026683,...,0.460279,0.306853,0.154033,5975.0,0.713808,0.213389,0.073640,1.38,26.2,22.4
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,52.396789,4.876607,21185.0,0.052396,...,0.438282,0.202974,0.235308,12640.0,0.592168,0.208070,0.199367,1.66,30.3,24.1
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,52.392305,4.855884,580.0,0.051724,...,0.387931,0.232759,0.146552,295.0,0.457627,0.389831,0.152542,1.74,35.0,28.5
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,52.379093,4.885109,14860.0,0.038694,...,0.363055,0.248654,0.114401,9850.0,0.659898,0.198477,0.141117,1.51,33.3,27.2


In [37]:
postcode.shape

(248, 81)

In [38]:
postcode.dtypes

postcode                    int64
wijk                       object
buurt                      object
woonplaats                 object
gemeente                   object
provincie                  object
latitude                  float64
longitude                 float64
tot_inwoners              float64
tot_0-5                   float64
tot_5-10                  float64
tot_10-15                 float64
tot_15-20                 float64
tot_20-25                 float64
tot_25-30                 float64
tot_30-35                 float64
tot_35-40                 float64
tot_40-45                 float64
tot_45-50                 float64
tot_50-55                 float64
tot_55-60                 float64
tot_60-65                 float64
tot_65-70                 float64
tot_70-75                 float64
tot_75-80                 float64
tot_80-85                 float64
tot_85-90                 float64
tot_90-95                 float64
tot_95-                   float64
tot_man       

In [39]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Alphen aan de Rijn, Netherlands') # Choice for Woerden as it centers on the Randstad area where our four selected cities are
NL_coords = g.latlng
NL_coords

[52.12743000000006, 4.658820000000048]

In [53]:
nl_map = folium.Map(location=[NL_coords[0], NL_coords[1]], zoom_start=9)

for lat, lng, pc4 in zip(postcode['latitude'], postcode['longitude'], postcode['postcode']):
    label = pc4
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(nl_map)  

# display map
nl_map

In [42]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Utrecht, Netherlands')
Utrecht_coords = g.latlng
print("Utrecht", Utrecht_coords)
g = geocoder.arcgis('Amsterdam, Netherlands')
amsterdam_coords = g.latlng
print("Amsterdam", amsterdam_coords)
g = geocoder.arcgis('Rotterdam, Netherlands')
rotterdam_coords = g.latlng
print("Rotterdam", rotterdam_coords)
g = geocoder.arcgis('Den Haag, Netherlands')
denhaag_coords = g.latlng
print("The Hague", denhaag_coords)

Utrecht [52.08965000000006, 5.114350000000059]
Amsterdam [52.36993000000007, 4.907880000000034]
Rotterdam [51.92282000000006, 4.478480000000047]
The Hague [52.08409000000006, 4.317320000000052]
